## Backtest 

In [2]:
import pandas as pd
import numpy as np

# Función para calcular las Bandas de Bollinger
def bollinger_bands(data, window=20, k=2):
    ma = data.rolling(window=window).mean()
    std = data.rolling(window=window).std()
    upper_band = ma + (std * k)
    lower_band = ma - (std * k)
    return upper_band, lower_band

# Función para calcular el Ratio de Sharpe
def sharpe_ratio(returns, risk_free_rate=0):
    if np.std(returns) == 0:
        return np.nan
    excess_returns = returns - risk_free_rate
    return np.mean(excess_returns) / np.std(excess_returns)

# Cálculo del Drawdown Máximo 
def calculate_max_drawdown(capital_series):
    capital_series = capital_series.dropna()

    # Calcular el rendimiento acumulado
    cumulative_returns = capital_series / capital_series.iloc[0]

    # Calcular el drawdown como la diferencia respecto al máximo acumulado
    drawdown = 1 - (cumulative_returns / cumulative_returns.cummax())
    
    # Retornar el máximo drawdown
    return drawdown.max()

# Cálculo de la razón Ganancias/Pérdidas
def win_loss_ratio(trades):
    wins = sum(1 for t in trades if t['action'] == 'sell' and t['price'] > trades[trades.index(t) - 1]['price'])
    losses = len([t for t in trades if t['action'] == 'sell']) - wins
    return wins / (losses if losses != 0 else 1)  # Evitamos división por cero

# Función para realizar el backtest de Bollinger
def backtest_bollinger(data, stop_losses, take_profits, window=20, k=2):
    sharpe_results = []
    win_loss_results = []
    max_drawdown_results = []
    combinations = []
    
    upper_band, lower_band = bollinger_bands(data, window, k)
    
    upper_band = upper_band.dropna()
    lower_band = lower_band.dropna()
    data = data[len(data) - len(upper_band):]

    for stop_loss in stop_losses:
        for take_profit in take_profits:
            returns = []
            trades = []  # Lista para almacenar las operaciones realizadas
            capital = 1.0
            position = 0
            buy_price = 0
            capital_series = []  # Para almacenar el capital a lo largo del tiempo (para drawdown)

            for i in range(len(upper_band)):
                if position == 0 and data.iloc[i] < lower_band.iloc[i]:  # Comprar
                    position = capital / data.iloc[i]
                    buy_price = data.iloc[i]
                    trades.append({'action': 'buy', 'price': buy_price, 'day': i})
                elif position > 0:
                    if data.iloc[i] <= buy_price * (1 - stop_loss):  # Stop Loss
                        capital = position * data.iloc[i]
                        position = 0
                        trades.append({'action': 'sell', 'price': data.iloc[i], 'day': i})
                    elif data.iloc[i] >= buy_price * (1 + take_profit):  # Take Profit
                        capital = position * data.iloc[i]
                        position = 0
                        trades.append({'action': 'sell', 'price': data.iloc[i], 'day': i})

                # Almacenamos el capital después de cada iteración para calcular el drawdown
                capital_series.append(capital)

                # Si la posición está cerrada (es decir, después de una venta), agregamos el rendimiento
                if position == 0:
                    returns.append(capital - 1)

            # Después del ciclo, calculamos las métricas
            if len(returns) > 0:
                sharpe_results.append(sharpe_ratio(np.array(returns)))
                win_loss_results.append(win_loss_ratio(trades))
                max_drawdown_results.append(calculate_max_drawdown(pd.Series(capital_series)))  # Usamos el capital para calcular el drawdown
            else:
                sharpe_results.append(np.nan)
                win_loss_results.append(np.nan)
                max_drawdown_results.append(np.nan)
            
            combinations.append(f"SL:{stop_loss} TP:{take_profit}")

    return sharpe_results, win_loss_results, max_drawdown_results, combinations

# Cargar los datos
data = pd.read_csv('synthetic_prices.csv')

# Eliminar la primera fila, que es un índice
data = data.iloc[1:, :]

# Transponer el DataFrame para que cada columna sea una lista de precios sinteticos
data = data.T 

# Asegurarse de que los índices sean secuenciales
data = data.reset_index(drop=True)

# Definir los niveles de Stop Loss y Take Profit
stop_losses = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]
take_profits = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]

# Inicializar los DataFrames para almacenar los resultados
sharpe_results_df = pd.DataFrame(columns=['Stop_Loss_Take_Profit'] + list(data.columns))
win_loss_results_df = pd.DataFrame(columns=['Stop_Loss_Take_Profit'] + list(data.columns))
max_drawdown_results_df = pd.DataFrame(columns=['Stop_Loss_Take_Profit'] + list(data.columns))

# Iterar sobre cada acción
for col in data.columns:
    action_data = data[col]
    sharpe_results = []
    win_loss_results = []
    max_drawdown_results = []
    combinations = []
    
    # Realizar el backtest para cada combinación de Stop Loss y Take Profit
    for stop_loss in stop_losses:
        for take_profit in take_profits:
            sharpe, win_loss, max_drawdown, comb = backtest_bollinger(action_data, [stop_loss], [take_profit])
            sharpe_results.append(sharpe[0])
            win_loss_results.append(win_loss[0])
            max_drawdown_results.append(max_drawdown[0])
            combinations.extend(comb)

    # Añadir los resultados al DataFrame
    sharpe_results_df[col] = sharpe_results
    win_loss_results_df[col] = win_loss_results
    max_drawdown_results_df[col] = max_drawdown_results
    sharpe_results_df['Stop_Loss_Take_Profit'] = combinations
    win_loss_results_df['Stop_Loss_Take_Profit'] = combinations
    max_drawdown_results_df['Stop_Loss_Take_Profit'] = combinations

    # Calcular el promedio de cada fila sin incluir la primera columna
    sharpe_results_df['Promedio'] = sharpe_results_df.iloc[:, 1:].mean(axis=1)
    win_loss_results_df['Promedio'] = win_loss_results_df.iloc[:, 1:].mean(axis=1)
    max_drawdown_results_df['Promedio'] = max_drawdown_results_df.iloc[:, 1:].mean(axis=1)

# Guardar los resultados en archivos CSV
sharpe_results_df.to_csv('sharpe_results.csv', index_label='Stop_Loss_Take_Profit')
win_loss_results_df.to_csv('win_loss_results.csv', index_label='Stop_Loss_Take_Profit')
max_drawdown_results_df.to_csv('max_drawdown_results.csv', index_label='Stop_Loss_Take_Profit')

# Mostrar los resultados
print("Sharpe Results:")
print(sharpe_results_df)
print("\nWin/Loss Ratio Results:")
print(win_loss_results_df)
print("\nMax Drawdown Results:")
print(max_drawdown_results_df)


Sharpe Results:
   Stop_Loss_Take_Profit         1         2         3         4         5  \
0        SL:0.01 TP:0.01  0.222984 -1.251401 -1.426059  0.857396  0.613860   
1        SL:0.01 TP:0.02  0.422170 -1.197728 -0.846458 -0.753724 -0.010045   
2        SL:0.01 TP:0.03  0.104748 -1.184330 -0.835500 -0.289296 -0.504031   
3        SL:0.01 TP:0.04 -1.604578 -1.312968 -0.352362 -2.468777 -0.379005   
4        SL:0.01 TP:0.05 -1.522918 -1.182130 -1.310264 -2.059659 -0.006370   
..                   ...       ...       ...       ...       ...       ...   
95        SL:0.1 TP:0.06 -1.107297 -0.991250 -1.574320 -1.491466  0.374671   
96        SL:0.1 TP:0.07 -1.087272 -0.991250 -1.565421 -1.409484 -0.022263   
97        SL:0.1 TP:0.08 -1.087272 -0.991250 -1.565421 -1.409484 -1.545081   
98        SL:0.1 TP:0.09 -1.087272 -0.991250 -1.513206 -2.093233 -1.545081   
99         SL:0.1 TP:0.1 -1.087272 -0.991250 -1.513206 -2.093233 -1.525037   

           6         7         8         9  ...

In [3]:
resultados_final=pd.DataFrame()
resultados_final["Stop_Loss_Take_Profit"]=sharpe_results_df["Stop_Loss_Take_Profit"]
resultados_final['Ratio de Sharpe']=sharpe_results_df["Promedio"]
resultados_final['Win-Loss Ratio']=win_loss_results_df["Promedio"]
resultados_final['Max Drawdown']=max_drawdown_results_df["Promedio"]

In [4]:
resultados_final

,Stop_Loss_Take_Profit,Ratio de Sharpe,Win-Loss Ratio,Max Drawdown
0,SL:0.01 TP:0.01,-0.488386,1.467295,0.095856
1,SL:0.01 TP:0.02,-0.703218,1.061303,0.116307
2,SL:0.01 TP:0.03,-0.900298,0.794912,0.134008
3,SL:0.01 TP:0.04,-1.019603,0.598058,0.154425
4,SL:0.01 TP:0.05,-1.144411,0.444788,0.168147
...,...,...,...,...
95,SL:0.1 TP:0.06,-1.450552,0.79786,0.335982
96,SL:0.1 TP:0.07,-1.513075,0.556455,0.351465
97,SL:0.1 TP:0.08,-1.518096,0.439776,0.364342
98,SL:0.1 TP:0.09,-1.574225,0.330674,0.371357


In [5]:
# 1. Aplanar las métricas de las 3 métricas (Sharpe, Win/Loss, Max Drawdown)
# Primero, asegurémonos de que estamos trabajando con los DataFrames correctos.
# Asumimos que flattened_df es un DataFrame que contiene todas las combinaciones de SL y TP
# con sus respectivas métricas por acción.

flattened_df = pd.DataFrame()

# 2. Crear el DataFrame con las combinaciones y métricas
for col in data.columns:
    action_sharpe = sharpe_results_df[col]
    action_win_loss = win_loss_results_df[col]
    action_max_drawdown = max_drawdown_results_df[col]
    combinations = sharpe_results_df['Stop_Loss_Take_Profit']
    
    # Aplanamos los resultados de las métricas y las combinaciones para cada acción
    temp_df = pd.DataFrame({
        'Combination': combinations,
        'Action': [col] * len(combinations),
        'Sharpe': action_sharpe,
        'Win_Loss_Ratio': action_win_loss,
        'Max_Drawdown': action_max_drawdown
    })
    
    # Añadimos este DataFrame al DataFrame general 'flattened_df'
    flattened_df = pd.concat([flattened_df, temp_df], ignore_index=True)

# 3. Calcular un 'Score' global para cada combinación (puedes modificar la fórmula según tus necesidades)
# Para simplificar, usaremos la fórmula: 
# Score = (Sharpe + Win_Loss_Ratio) - Max_Drawdown
# Asegúrate de que las métricas no sean NaN antes de calcular el Score.

flattened_df['Score'] = (flattened_df['Sharpe'].fillna(0) + flattened_df['Win_Loss_Ratio'].fillna(0)) - flattened_df['Max_Drawdown'].fillna(0)

# 4. Asegurémonos de que la columna 'Combination' esté bien formateada antes de extraer SL y TP
# Verifica las primeras filas para asegurar que las combinaciones sean cadenas como 'SL:<valor> TP:<valor>'
print(flattened_df['Combination'].head())

# 5. Extraemos los valores de 'Stop_Loss' y 'Take_Profit' de la columna 'Combination'
flattened_df['Stop_Loss'] = flattened_df['Combination'].apply(lambda x: float(str(x).split("SL:")[1].split()[0]) if isinstance(x, str) else np.nan)
flattened_df['Take_Profit'] = flattened_df['Combination'].apply(lambda x: float(str(x).split("TP:")[1].split()[0]) if isinstance(x, str) else np.nan)

# 6. Ordenamos todas las combinaciones globalmente según el 'Score' de mejor a peor
top_10_combinations = flattened_df.sort_values(by='Score', ascending=False).head(10)

# 7. Mostrar las 10 mejores combinaciones
print("\nTop 10 combinaciones globales:")
print(top_10_combinations[['Stop_Loss', 'Take_Profit', 'Score']])

# Asumiendo que 'flattened_df' es el DataFrame con las combinaciones, las métricas y las puntuaciones combinadas
# Asumiendo que 'flattened_df' es el DataFrame con las combinaciones, las métricas y las puntuaciones combinadas

# Paso 1: Eliminar combinaciones duplicadas (mismo Stop Loss y Take Profit)
unique_combinations_df = flattened_df.drop_duplicates(subset=['Stop_Loss', 'Take_Profit'])

# Paso 2: Ordenar el DataFrame por el 'Score' de mayor a menor
top_combinations_sorted = unique_combinations_df.sort_values(by='Score', ascending=False)

# Paso 3: Seleccionar las 10 mejores combinaciones
top_10_combinations = top_combinations_sorted.head(10)

# Paso 4: Mostrar las 10 mejores combinaciones
print("Top 10 combinaciones globales:")
print(top_10_combinations[['Stop_Loss', 'Take_Profit', 'Score']])

# Opcional: Si deseas guardar los resultados en un archivo CSV
top_10_combinations.to_csv('top_10_combinations.csv', index=False)




0    SL:0.01 TP:0.01
1    SL:0.01 TP:0.02
2    SL:0.01 TP:0.03
3    SL:0.01 TP:0.04
4    SL:0.01 TP:0.05
Name: Combination, dtype: object

Top 10 combinaciones globales:
      Stop_Loss  Take_Profit      Score
1770       0.08         0.01  11.418594
1780       0.09         0.01  11.417336
1740       0.05         0.01  11.413936
1760       0.07         0.01  11.413936
1750       0.06         0.01  11.413936
1490       0.10         0.01  11.388699
1480       0.09         0.01  11.388699
1420       0.03         0.01  11.388609
1440       0.05         0.01  11.388609
1470       0.08         0.01  11.388609
Top 10 combinaciones globales:
    Stop_Loss  Take_Profit     Score
1        0.01         0.02  1.735222
0        0.01         0.01  1.536036
2        0.01         0.03  0.989866
22       0.03         0.03  0.505853
21       0.03         0.02  0.063307
90       0.10         0.01 -0.143517
91       0.10         0.02 -0.218699
92       0.10         0.03 -0.250693
80       0.09         0.01

In [6]:
# Ordenamos todas las combinaciones globalmente según el 'Score' de mejor a peor
top_10_combinations = flattened_df.sort_values(by='Score', ascending=False).drop_duplicates(subset=['Stop_Loss', 'Take_Profit']).head(10)

# Mostrar las 10 mejores combinaciones
print("\nTop 10 combinaciones globales:")
print(top_10_combinations[['Stop_Loss', 'Take_Profit', 'Score']])



Top 10 combinaciones globales:
      Stop_Loss  Take_Profit      Score
1770       0.08         0.01  11.418594
1780       0.09         0.01  11.417336
1740       0.05         0.01  11.413936
1760       0.07         0.01  11.413936
1750       0.06         0.01  11.413936
1490       0.10         0.01  11.388699
1420       0.03         0.01  11.388609
1430       0.04         0.01  11.388609
1441       0.05         0.02  11.350312
1471       0.08         0.02  11.350312


In [7]:
# DataFrame de combinaciones seleccionadas (los Top 10 globales)
top_10_combinations = top_10_combinations[['Stop_Loss', 'Take_Profit']].reset_index(drop=True)
# Crear un nuevo DataFrame vacío para almacenar los resultados
metrics_df = pd.DataFrame(columns=['Sharpe', 'Win_Loss_Ratio', 'Max_Drawdown'])

#Buscar las combinaciones en los DataFrames de métricas y extraer el valor de la columna 'Promedio'

# Iterar sobre las 10 mejores combinaciones globales
for _, comb in top_10_combinations.iterrows():
    # Extraer la combinación de Stop Loss y Take Profit
    stop_loss = comb['Stop_Loss']
    take_profit = comb['Take_Profit']
    
    # Asegurarnos de que la combinación se formatee correctamente (por ejemplo, SL:0.08 TP:0.01)
    combination_str = f"SL:{stop_loss} TP:{take_profit}"
    

    # Buscar en el DataFrame de Sharpe
    sharpe_value = sharpe_results_df[sharpe_results_df['Stop_Loss_Take_Profit'] == combination_str]['Promedio'].values
    win_loss_value = win_loss_results_df[win_loss_results_df['Stop_Loss_Take_Profit'] == combination_str]['Promedio'].values
    max_drawdown_value = max_drawdown_results_df[max_drawdown_results_df['Stop_Loss_Take_Profit'] == combination_str]['Promedio'].values
    
    # Comprobar si los valores existen
    if sharpe_value.size > 0 and win_loss_value.size > 0 and max_drawdown_value.size > 0:
        # Añadir los resultados de la combinación al nuevo DataFrame
        metrics_df.loc[combination_str] = [sharpe_value[0], win_loss_value[0], max_drawdown_value[0]]
    else:
        print(f"No se encontraron datos para la combinación: {combination_str}")

# Paso 4: Mostrar el nuevo DataFrame
print("\nDataFrame con las métricas de las mejores combinaciones:")
print(metrics_df)

# Paso 5: Guardar el DataFrame en un archivo CSV (opcional)
metrics_df.to_csv('best_combinations_metrics.csv')



DataFrame con las métricas de las mejores combinaciones:
                   Sharpe  Win_Loss_Ratio  Max_Drawdown
SL:0.08 TP:0.01 -1.049747        3.266691      0.192766
SL:0.09 TP:0.01 -1.088977        3.307940      0.208439
SL:0.05 TP:0.01 -0.928887        2.804827      0.154608
SL:0.07 TP:0.01 -1.047666        3.140848      0.179239
SL:0.06 TP:0.01 -0.966341        2.938678      0.168592
SL:0.1 TP:0.01  -1.174888        3.368049      0.220935
SL:0.03 TP:0.01 -0.953400        2.132099      0.131870
SL:0.04 TP:0.01 -0.973853        2.503297      0.141233
SL:0.05 TP:0.02 -1.115616        1.844925      0.187571
SL:0.08 TP:0.02 -1.208794        2.184601      0.232504
